In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import random

from dataset import Thyroid_Dataset
from model import Eff_Unet
from loss_metric import DiceLoss, IOU_score

C:\Users\oplab\anaconda3\envs\thyroid\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\oplab\anaconda3\envs\thyroid\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
import wandb
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: s960068sss (s960068sss-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
image_size = 128
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])
train_dataset = Thyroid_Dataset("train.csv", transform = transform, augmentation = True)
train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True)

val_dataset = Thyroid_Dataset("val.csv", transform = transform)
val_dataloader = DataLoader(val_dataset, batch_size = 8)
# image, mask = next(iter(train_dataloader))
# image = image.cuda()
# mask = mask.cuda()

In [5]:
# model = Eff_Unet(
#         layers=[5, 5, 15, 10],
#         embed_dims=[40, 80, 192, 384],
#         downsamples=[True, True, True, True],
#         vit_num=6,
#         drop_path_rate=0.1,
#         num_classes=1,
#         resolution = image_size).cuda()


In [6]:
# loss_fn = DiceLoss()

In [7]:
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# for epoch in range(1000):
#     outputs = model(image)
#     # outputs = torch.sigmoid(logits)
#     loss = loss_fn(outputs, mask)
#     IOU = IOU_score(outputs, mask)

#     # Backward and optimize
#     optimizer.zero_grad()   # clear previous gradients
#     loss.backward()         # compute gradients
#     optimizer.step()        # update weights

#     print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, IOU: {IOU.item():.4f}")

In [9]:
def train(dataloader, model, optimizer, loss_fn, device):
    total_loss = 0
    total_IOU = 0
    
    model.train()
    model.to(device)
    for image, mask in tqdm(dataloader):
        image, mask = image.to(device), mask.to(device)
        outputs = model(image)
        
        loss = loss_fn(outputs, mask)
        IOU = IOU_score(outputs, mask)
    
        # Backward and optimize
        optimizer.zero_grad()   # clear previous gradients
        loss.backward()         # compute gradients
        optimizer.step()        # update weights

        total_loss += loss.item()
        total_IOU += IOU.item()
    return total_loss/len(dataloader), total_IOU/len(dataloader)
        
def val(dataloader, model, loss_fn, device):
    total_loss = 0
    total_IOU = 0

    model.eval()
    model.to(device)
    for image, mask in tqdm(dataloader):
        image, mask = image.to(device), mask.to(device)
        outputs = model(image)
        
        loss = loss_fn(outputs, mask)
        IOU = IOU_score(outputs, mask)
    

        total_loss += loss.item()
        total_IOU += IOU.item()
    return total_loss/len(dataloader), total_IOU/len(dataloader)

In [10]:
wandb.init(
    project="thyroid",  # Project name in W&B
    name="augmentation v1",       # (optional) specific run name
    config={                     # (optional) hyperparameters
        "image_size": 128,
        "learning_rate": 0.001,
        "epochs": 20,
        "batch_size": 8
    }
)

In [11]:
model = Eff_Unet(
        layers=[5, 5, 15, 10],
        embed_dims=[40, 80, 192, 384],
        downsamples=[True, True, True, True],
        vit_num=6,
        drop_path_rate=0.1,
        num_classes=1,
        resolution = image_size).cuda()


In [12]:
loss_fn = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
for epoch in range(20):
    print(f"epoch : {epoch}")
    total_loss_train, total_IOU_train = train(train_dataloader, model, optimizer, loss_fn, "cuda")
    print(f"train loss : {total_loss_train}, train IOU : {total_IOU_train}")
    total_loss_val, total_IOU_val = val(val_dataloader, model, loss_fn, "cuda")
    print(f"val loss : {total_loss_val}, val IOU : {total_IOU_val}")

    wandb.log({
        "epoch": epoch,
        "train_loss": total_loss_train,
        "train_IOU": total_IOU_train,
        "val_loss": total_loss_val,
        "val_IOU": total_IOU_val
    })
wandb.finish()

epoch : 0


 11%|█████████                                                                        | 83/747 [00:21<02:51,  3.88it/s]


KeyboardInterrupt: 

In [ ]:
# total_loss_train, total_IOU_train = train(train_dataloader, model, optimizer, loss_fn, "cuda")

In [ ]:
# total_loss_val, total_IOU_val = val(val_dataloader, model, loss_fn, "cuda")

In [17]:
print("ll")

ll
